In [ ]:
import pandas as pd    
import numpy as np 

import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.charts import Line,Bar,Map,Pie,Grid,Tab,Timeline

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB 


df_confirmed = pd.read_csv('data/china_confirmed.csv', header=0, index_col=0)
df_death = pd.read_csv('data/china_death.csv', header=0, index_col=0)
df_recovery = pd.read_csv('data/china_recovery.csv', header=0, index_col=0)
 
# summary by date
sum_confirmed = df_confirmed.sum(axis=1)
sum_death = df_death.sum(axis=1)
sum_recovery = df_recovery.sum(axis=1)
# summary by distric
sum_dist_confirmed = df_confirmed.sum(axis=0)
sum_dist_death = df_death.sum(axis=0)
sum_dist_recovery = df_recovery.sum(axis=0)


accum_confirmed = sum_confirmed.cumsum()
accum_death = sum_death.cumsum()
accum_recovery = sum_recovery.cumsum()
  
 

In [ ]:
 
line_daily = Line(init_opts=opts.InitOpts(theme=ThemeType.DARK))
line_daily.add_xaxis(sum_confirmed.index.tolist())
line_daily.add_yaxis('每日确诊',sum_confirmed.values.tolist(),label_opts=opts.LabelOpts(is_show=False))
line_daily.add_yaxis('每日恢复',sum_recovery.values.tolist(),label_opts=opts.LabelOpts(is_show=False))
line_daily.add_yaxis('每日死亡',sum_death.values.tolist(),label_opts=opts.LabelOpts(is_show=False))
 

line_cum = Line(init_opts=opts.InitOpts(theme=ThemeType.DARK))
line_cum.add_xaxis(sum_confirmed.index.tolist())
line_cum.add_yaxis('累计确诊',accum_confirmed.values.tolist(),label_opts=opts.LabelOpts(is_show=False))
line_cum.add_yaxis('累计恢复',accum_recovery.values.tolist(),label_opts=opts.LabelOpts(is_show=False))
line_cum.add_yaxis('累计死亡',accum_death.values.tolist(),label_opts=opts.LabelOpts(is_show=False))


pie = Pie(init_opts=opts.InitOpts(theme=ThemeType.DARK))
pie_data = [(sum_dist_confirmed.index[i],int(sum_dist_confirmed.values[i])) for i in range(len(sum_dist_confirmed))]
pie.add('确诊病例分布', data_pair=pie_data)
pie.set_global_opts(legend_opts=opts.LegendOpts(is_show=False)) 

pie_death = Pie(init_opts=opts.InitOpts(theme=ThemeType.DARK))
pie_death_data = [(sum_dist_death.index[i],int(sum_dist_death.values[i])) for i in range(len(sum_dist_death))]
pie_death.add('死亡病例分布', data_pair=pie_death_data)
pie_death.set_global_opts(legend_opts=opts.LegendOpts(is_show=False))



In [ ]:
visual_opts = opts.VisualMapOpts(max_=100000, is_piecewise=True,
                                 pieces=[
                                     {"max": 100000, "min": 10001,
                                      "label": ">1000", "color": "#6A0C0C"},
                                     {"max": 10000, "min": 1001,
                                      "label": ">1000", "color": "#8A0808"},
                                     {"max": 1000, "min": 500,
                                      "label": "500-1000", "color": "#FF0000"},
                                     {"max": 499, "min": 100,
                                      "label": "100-499", "color": "#B8860B"},
                                     {"max": 99, "min": 10,
                                      "label": "10-99", "color": "#EEB422"},
                                     {"max": 9, "min": 1, "label": "1-9",
                                      "color": "#EEDD82"},
                                     {"max": 0, "min": 0, "label": "0",
                                      "color": "#FFFFFF"},
                                 ])

all_data = df_confirmed.cumsum()
tl = Timeline()
for day in all_data.index:  
    s = (all_data.loc[day])
    pairlst = [(s.index[i],int(s.values[i])) for i in range(len(s))]
    map0 = (
        Map(init_opts=opts.InitOpts(theme=ThemeType.DARK))
        .add("累计确诊",pairlst, "china")
        .set_global_opts(visualmap_opts=visual_opts) 
    )
    tl.add(map0, day) 
    
 

In [ ]:
tab =  Tab()
tab.add(tl,"Covid-19地图")
tab.add(line_daily,"每日新增")
tab.add(line_cum,"累计病例")
tab.add(pie,"各省确诊分布")
tab.add(pie_death,"各省死亡病例分布")

tab.load_javascript()

In [ ]:
tab.render_notebook() 